### Traveling salesman problem solved using Simulated Annealing

In [1]:
from scipy import *
from numpy import *
from pylab import *
from geopy.geocoders import Nominatim
import random
import matplotlib.pyplot as plt

In [ ]:
def readCities():
    P = []
    PNames = []

    # get coordinates of cities
    geolocator = Nominatim(user_agent='kunal_APP')
    with open(r"cities_name.txt", 'r') as file:
        for line in file:
            city = line.rstrip()
            if city == "":
                break

            pt = geolocator.geocode(city, timeout=10000)
            print(f"City = {city}, latitude = {pt.latitude}, longitude = {pt.longitude}")
            P.append([pt.latitude, pt.longitude])
            PNames.append(city)
    P = array(P)
    return P, PNames

In [ ]:
P, PNames = readCities()
print('no of cities =', len(P))
print(PNames)

In [ ]:
def Distance(P1, P2):
    if array_equal(P1, P2):
        return 0.0
    
    d = sqrt((P1[0] - P2[0])**2 + (P1[1] - P2[1])**2)
    return d

In [ ]:
def TotalDistance(P, seq):
    dist = 0.0
    N = len(seq)
    for i in range(N-1):
        dist = dist + Distance(P[seq[i]], P[seq[i+1]])

    dist = dist + Distance(P[seq[N-1]], P[seq[0]])

    return dist

In [ ]:
def plot_fun(seq, P, dist, PNames, title):
    Pt = [P[seq[i]] for i in range(len(seq))]
    Pt += [P[seq[0]]]
    Pt = array(Pt)
    plt.title(title)
    plt.plot(Pt[:, 0], Pt[:, 1], '-o')

    for i in range(len(P)):
        plt.annotate(PNames[i], (P[i][0], P[i][1]))
    plt.show()

In [ ]:
def swap(P, seq, dist, N1, N2, temp, nCity, PNames):
    N1L = N1 - 1
    if N1L < 0:
        N1L = N1L + nCity
    N1R = N1 + 1
    if N1R >= nCity:
        N1R = 0
    N2L = N2 - 1
    if N2L < 0:
        N2L = N2L + nCity
    N2R = N2 + 1 
    if N2R >= nCity:
        N2R = 0

    I1 = seq[N1]
    I2 = seq[N2]
    I1L = seq[N1L]
    I1R = seq[N1R]
    I2L = seq[N2L]
    I2R = seq[N2R]

    delta = 0.0
    delta = delta + Distance(P[I1L], P[I2])
    delta = delta + Distance(P[I1], P[I2R])
    delta = delta - Distance(P[I1L], P[I1])
    delta = delta - Distance(P[I2], P[I2R])

    if N1 != N2L and N1R != N2 and N1R != N2L and N2 != N1L:
        delta += Distance(P[I2], P[I1R])
        delta += Distance(P[I2L], P[I1])
        delta -= Distance(P[I1], P[I1R])
        delta -= Distance(P[I1L], P[I2])

    prob = 1.0
    if delta > 0.0:
        prob = exp(-delta / temp)

    rndm = random.random()

    if rndm < prob:
        dist += delta

        seq[N1] = I2
        seq[N2] = I1

        dif = abs(dist - TotalDistance(P, seq))
        if dif * dif > 0.01:
            print(f"in SWAP N1={N1} N2={N2} N1L={N1L} N1R={N1R} N2L={N2L} N2R={N2R}")
            print(f"I1={I1} I2={I2} I1L={I1L} I1R={I1R} I2L={I2L} I2R={I2R}")
            print(f"T={temp:.6f} D={dist:.6f} delta={delta:.6f} p={prob:.6f} rn={rndm:.6f}")
            print(seq)
            print()

            # Plot the current state of the route
            plt.clf()
            plot_fun(seq, P, dist, PNames, f"Swap Iteration - T={temp:.2f}")
            plt.pause(0.1)  # Pause to allow the plot to render

        return dist, True
    else:
        return dist, False

In [ ]:
def reverse(P, seq, dist, N1, N2, temp, nCity, PNames):
    N1L = N1 - 1
    if N1L < 0:
        N1L += nCity

    N2R = N2 + 1
    if N2R >= nCity:
        N2R = 0

    delta = 0.0
    if (N1 != N2R) and (N2 != N1L):
        delta = Distance(P[seq[N1L]], P[seq[N2]]) \
                + Distance(P[seq[N1]], P[seq[N2R]]) \
                - Distance(P[seq[N1L]], P[seq[N1]]) \
                - Distance(P[seq[N2]], P[seq[N2R]])
    else:
        return dist, False
    
    prob = 1.0
    if delta > 0.0:
        prob = exp(-delta / temp)

    rndm = random.random()

    if rndm < prob:
        dist += delta

        i = N1
        j = N2 

        while i < j:
            u = seq[i] 
            seq[i] = seq[j]
            seq[j] = u
            i += 1
            j -= 1

        dif = abs(dist - TotalDistance(P, seq))
        if dif * dif > 0.01:
            print(f"in REVERSE N1L={N1L} N2R={N2R}")
            print(f"N1={N1} N2={N2} T={temp:.6f} D={dist:.6f} delta={delta:.6f} p={prob:.6f} rn={rndm:.6f}")
            print(seq)
            print()

            # Plot the current state of the route
            plt.clf()
            plot_fun(seq, P, dist, PNames, f"Reverse Iteration - T={temp:.2f}")
            plt.pause(0.1)  # Pause to allow the plot to render

        return dist, True
    else:
        return dist, False

In [ ]:
def main():
    P, PNames = readCities()
    nCity = len(P)              # Number of cities to visit

    maxTsteps = 250             # Number of temperature steps
    fCool = 0.9                 # Cooling factor
    maxSwaps = 2000             # Number of swaps per temperature step
    maxAccepted = 10 * nCity    # Number of accepted steps at constant temperature

    seq = arange(0, nCity, 1)
    dist = TotalDistance(P, seq)
    temp = 10.0 * dist          # Initial temperature

    print("\n\n")
    print(seq)
    print(f"\n nCity={nCity:3d} dist={dist:.6f} temp={temp:.6f} \n")
    input("Press Enter to continue...")

    plt.ion()
    plot_fun(seq, P, dist, PNames, "Initial Route")
    plt.pause(1)  # Pause to allow plot to render

    oldDist = 0.0
    convergenceCount = 0

    for t in range(1, maxTsteps + 1):
        if temp < 1.0e-6:
            break

        accepted = 0
        iteration = 0

        while iteration <= maxSwaps and accepted < maxAccepted:
            N1 = -1
            while N1 < 0 or N1 >= nCity:
                N1 = int(random.random() * nCity)

            N2 = -1
            while N2 < 0 or N2 >= nCity or N2 == N1:
                N2 = int(random.random() * nCity)

            if N2 < N1:
                N1, N2 = N2, N1  # Swap N1 and N2

            chk = random.uniform(0, 1)
            if chk < 0.5 and N1 + 1 != N2:
                dist, flag = swap(P, seq, dist, N1, N2, temp, nCity, PNames)
            else:
                dist, flag = reverse(P, seq, dist, N1, N2, temp, nCity, PNames)

            if flag:
                accepted += 1

            iteration += 1

        print(f"Iteration: {t} temp={temp:.6f} dist={dist:.6f}")
        print("seq= ")
        set_printoptions(precision=3)
        print(seq)
        print()

        if abs(dist - oldDist) < 1.0e-4:
            convergenceCount += 1
        else:
            convergenceCount = 0

        if convergenceCount >= 4:
            break

        if t % 25 == 0:
            plt.clf()
            plot_fun(seq, P, dist, PNames, f"Iteration {t} Route")
            plt.pause(0.1)

        temp *= fCool
        oldDist = dist

    plt.ioff()
    plt.clf()
    plot_fun(seq, P, dist, PNames, "Final Route")
    plt.show()

if __name__ == '__main__':
    main()